In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from datetime import datetime

In [ ]:
def plot_metric(week_dates, metric, legends, title): 
    to_plot_list=[]
    to_plot_list.append(week_dates)
    metric_size_per_week_per_policy = report_df[metric].values
    for i in metric_size_per_week_per_policy:
        to_plot_list.append(i)
        
    to_plot_df = pd.DataFrame(to_plot_list)
    to_plot_df_T = to_plot_df.T
    to_plot_df_T.columns= legends
    plt.figure(figsize=(20,10))
  
    for i in range(1, len(legends)):
        plt.plot( 'date', legends[i], data=to_plot_df_T)
    plt.legend()
    plt.xticks(week_dates, rotation=45)

In [ ]:
# Read the report
report_df= pd.read_parquet("/Users/ddavila/projects/DOMA/data/model/reports/072018_062019_p1_3")

# Get a list of human readable dates from the timestmps on the report DataFrame
# to be used for the xticks 
week_dates= []
for week_ts in report_df.weeks_ts.values[0]:
    #print(week_ts)
    week_date = datetime.utcfromtimestamp(week_ts).strftime('%Y-%m-%d')
    week_dates.append(week_date)
    
# Create the legends list
legends=["date"]
for i in report_df.policy.values:
    legend = str(i)+"week"
    legends.append(legend)

In [ ]:
plot_metric(week_dates, "ws_size_per_week_per_policy", legends, "Workingset size per week")

In [ ]:
plot_metric(week_dates, "recalled_size_per_week_per_policy", legends, "Recalled size per week")

In [ ]:
plot_metric(week_dates, "freed_size_per_week_per_policy", legends, "Freed size per week")